## Store Sales - Time Series Forecasting

In [ ]:
## Linear Regression

In [ ]:
# imports 
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import plotly.express as px
import statsmodels.api as sm
import seaborn as sns
import gc

In [ ]:
# data
train_df= pd.read_csv("/Users/krushna/Downloads/store-sales-time-series-forecasting/train.csv")
test_df = pd.read_csv("/Users/krushna/Downloads/store-sales-time-series-forecasting/test.csv")
stores_df = pd.read_csv("/Users/krushna/Downloads/store-sales-time-series-forecasting/stores.csv")
transactions_df = pd.read_csv("/Users/krushna/Downloads/store-sales-time-series-forecasting/transactions.csv")
train_df=train_df.sort_values(['store_nbr', 'date'])
transactions_df=transactions_df.sort_values(['store_nbr', 'date'])

In [ ]:
train_df['date'] = pd.to_datetime(train_df['date'])
train_df.head()

In [ ]:
#formating
train_df.onpromotion = train_df.onpromotion.astype("float16")
train_df.sales = train_df.sales.astype("float32")
stores_df.cluster = stores_df.cluster.astype("int8")

In [ ]:
transactions_df["date"] = pd.to_datetime(transactions_df.date)
transactions_df.head()

In [ ]:
# Feature Engineering
temp = pd.merge(train_df.groupby(['date', 'store_nbr']).sales.sum().reset_index(), transactions_df, how = "left")

In [ ]:
# Data Visualization
print("Spearman Correlation Between Totals sales and Transactions: {:,.4f}".format(temp.corr('spearman').sales.loc["transactions"]))

In [ ]:
# Data Visualize 
px.line(transactions_df.sort_values(["store_nbr", "date"]), x='date', y='transactions', color='store_nbr',title = "Transactions" )

#### From this we can say that the transactions was maximum by the end of every year.

#

In [ ]:
# Check transactions for every month 
a = transactions_df.copy()
a['date'] = pd.to_datetime(a['date'])
a['year'] = a.date.dt.year
a['month'] = a.date.dt.month
px.box(a, x="year", y="transactions" , color = "month", title = "Transactions")

#### This confirms that the transactions is made he most in the month of december

In [ ]:
# Let's check weekly 
b = transactions_df.copy()
b['date'] = pd.to_datetime(a['date'])
b['day'] = b.date.dt.day
b['month'] = b.date.dt.month
b['year'] = b.date.dt.year

# Create a box plot for transactions with days on the x-axis
px.box(b, x="day", y="transactions", color="month", title="Transactions by Day")

#### This shows that the transactions is more in last week of the month.

In [ ]:
px.scatter(temp, x = "transactions", y = "sales", trendline = "ols", trendline_color_override = "red")

#### There is strong correlation between total sales and transactions

#

In [ ]:
# Now let's use the next dataset and see how it is effective 
oil_df= pd.read_csv("/Users/krushna/Downloads/store-sales-time-series-forecasting/oil.csv")
# Converting datetime 
oil_df['date'] = pd.to_datetime(oil_df.date)
# Resample
oil_df = oil_df.set_index("date").dcoilwtico.resample("D").sum().reset_index()
# Linear Interpolation 
oil_df["dcoilwtico"] = np.where(oil_df["dcoilwtico"] == 0, np.nan, oil_df["dcoilwtico"])
oil_df["dcoilwtico_interpolated"] =oil_df.dcoilwtico.interpolate()

In [ ]:
# Data Visualization
p = oil_df.melt(id_vars=['date']+list(oil_df.keys()[5:]), var_name='Legend')
px.line(p.sort_values(["Legend", "date"], ascending = [False, True]), x='date', y='value', color='Legend',title = "Daily Oil Price" )

#### We see that the coreelation is appropate but the curve is negative.Now lets check the Correlation of oil price and transactions and oil price and sales

In [ ]:
temp = pd.merge(temp, oil_df, how = "left")
print("Correlation with Daily Oil Prices")
print(temp.drop(["store_nbr", "dcoilwtico"], axis = 1).corr("spearman").dcoilwtico_interpolated.loc[["sales", "transactions"]], "\n")


fig, axes = plt.subplots(1, 2, figsize = (15,5))
temp.plot.scatter(x = "dcoilwtico_interpolated", y = "transactions", ax=axes[0])
temp.plot.scatter(x = "dcoilwtico_interpolated", y = "sales", ax=axes[1], color = "r")
axes[0].set_title('Daily oil price & Transactions', fontsize = 15)
axes[1].set_title('Daily Oil Price & Sales', fontsize = 15);

#### From this we can assume that the oil prices do make sense and we cannot ignore the dataset. As the oil prices increases the cost in Ecuador increases and economy goes down and due to that there are less sales.

#

In [ ]:
## Sales - Trget Variable 

# Looking for correlation matrix 
a = train_df[["store_nbr", "sales"]]
a["ind"] = 1
a["ind"] = a.groupby("store_nbr").ind.cumsum().values
a = pd.pivot(a, index = "ind", columns = "store_nbr", values = "sales").corr()
mask = np.triu(a.corr())
plt.figure(figsize=(20, 20))
sns.heatmap(a,
        annot=True,
        fmt='.1f',
        cmap='coolwarm',
        square=True,
        mask=mask,
        linewidths=1,
        cbar=False)
plt.title("Correlations among stores",fontsize = 20)
plt.show()

In [ ]:
## Now lets consider the store number equivalent as well as the sales with dates
a = train_df.set_index("date").groupby("store_nbr").resample("D").sales.sum().reset_index()
px.line(a, x = "date", y= "sales", color = "store_nbr", title = "Daily total sales of the stores")

In [ ]:
## This is a 3-D Data so after clicking each store number 
## there we find a need to remove some str_nbr whcih had least 
## sales an can harm our model.
print(train_df.shape)
train_df = train_df[~((train_df.store_nbr == 52) & (train_df.date < "2017-04-20"))]
train_df= train_df[~((train_df.store_nbr == 22) & (train_df.date < "2015-10-09"))]
train_df= train_df[~((train_df.store_nbr == 42) & (train_df.date < "2015-08-21"))]
train_df = train_df[~((train_df.store_nbr == 21) & (train_df.date < "2015-07-24"))]
train_df= train_df[~((train_df.store_nbr == 29) & (train_df.date < "2015-03-20"))]
train_df = train_df[~((train_df.store_nbr == 20) & (train_df.date < "2015-02-13"))]
train_df = train_df[~((train_df.store_nbr == 53) & (train_df.date < "2014-05-29"))]
train_df = train_df[~((train_df.store_nbr == 36) & (train_df.date < "2013-05-09"))]
train_df.shape

In [ ]:
c = train_df.groupby(["store_nbr", "family"]).sales.sum().reset_index().sort_values(["family","store_nbr"])
c = c[c.sales == 0]
c

#### The table above shows the list of family and their sales with 0 valuation

#

In [ ]:
print(train_df.shape)
# Anti Join
outer_join = train_df.merge(c[c.sales == 0].drop("sales",axis = 1), how = 'outer', indicator = True)
train_df = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis = 1)
del outer_join
gc.collect()
train_df.shape

In [ ]:
## Zero- Prediction
zero_prediction = []
for i in range(0,len(c)):
    zero_prediction.append(
        pd.DataFrame({
            "date":pd.date_range("2017-08-16", "2017-08-31").tolist(),
            "store_nbr":c.store_nbr.iloc[i],
            "family":c.family.iloc[i],
            "sales":0
        })
    )
zero_prediction = pd.concat(zero_prediction)
del c
gc.collect()
zero_prediction

In [ ]:
c = train_df.groupby(["family", "store_nbr"]).tail(60).groupby(["family", "store_nbr"]).sales.sum().reset_index()
c[c.sales == 0]

In [ ]:
#### Family like Babay Care, SCHOOL AND OFFICE SUPPLIES, Books
#### Lawn and Gardens, Ladieswear, have no sales.
#### Data Visualzization 

fig, ax = plt.subplots(1,5, figsize = (20,4))
train_df[(train_df.store_nbr == 10) & (train_df.family == "LAWN AND GARDEN")].set_index("date").sales.plot(ax = ax[0], title = "STORE 10 - LAWN AND GARDEN")
train_df[(train_df.store_nbr == 36) & (train_df.family == "LADIESWEAR")].set_index("date").sales.plot(ax = ax[1], title = "STORE 36 - LADIESWEAR")
train_df[(train_df.store_nbr == 6) & (train_df.family == "SCHOOL AND OFFICE SUPPLIES")].set_index("date").sales.plot(ax = ax[2], title = "STORE 6 - SCHOOL AND OFFICE SUPPLIES")
train_df[(train_df.store_nbr == 14) & (train_df.family == "BABY CARE")].set_index("date").sales.plot(ax = ax[3], title = "STORE 14 - BABY CARE")
train_df[(train_df.store_nbr == 53) & (train_df.family == "BOOKS")].set_index("date").sales.plot(ax = ax[4], title = "STORE 43 - BOOKS")
plt.show()

In [ ]:
#### Check all families 

a = train_df.set_index("date").groupby("family").resample("D").sales.sum().reset_index()
px.line(a, x = "date", y= "sales", color = "family", title = "Daily total sales of the family")

In [ ]:
a = train_df.groupby("family").sales.mean().sort_values(ascending = False).reset_index()
px.bar(a, y = "family", x="sales", color = "family", title = "Which product family preferred more?")

In [ ]:
## Let's check How the sales changes over cities 

d = pd.merge(train_df, stores_df)
d["store_nbr"] = d["store_nbr"].astype("int8")
d["year"] = d.date.dt.year
px.line(d.groupby(["city", "year"]).sales.mean().reset_index(), x = "year", y = "sales", color = "city")

In [ ]:
## Lets take a look for next Dataset - Holidays and Events

holidays_df = pd.read_csv("/Users/krushna/Downloads/store-sales-time-series-forecasting/holidays_events.csv")
holidays_df.head()


In [ ]:
import numpy as np
import pandas as pd

def create_date_features(df):
    df['month'] = df.date.dt.month.astype("int8")
    df['day_of_month'] = df.date.dt.day.astype("int8")
    df['day_of_year'] = df.date.dt.dayofyear.astype("int16")
    df['week_of_month'] = (df.date.apply(lambda d: (d.day-1)//7+1)).astype("int8")
    
    # Updated line for week_of_year
    df['week_of_year'] = df.date.apply(lambda x: x.isocalendar()[1]).astype("int8")

    df['day_of_week'] = (df.date.dt.dayofweek + 1).astype("int8")
    df['year'] = df.date.dt.year.astype("int32")
    df["is_wknd"] = (df.date.dt.weekday // 4).astype("int8")
    df["quarter"] = df.date.dt.quarter.astype("int8")
    df['is_month_start'] = df.date.dt.is_month_start.astype("int8")
    df['is_month_end'] = df.date.dt.is_month_end.astype("int8")
    df['is_quarter_start'] = df.date.dt.is_quarter_start.astype("int8")
    df['is_quarter_end'] = df.date.dt.is_quarter_end.astype("int8")
    df['is_year_start'] = df.date.dt.is_year_start.astype("int8")
    df['is_year_end'] = df.date.dt.is_year_end.astype("int8")
    
    # Season calculation
    df["season"] = np.where(df.month.isin([12, 1, 2]), 0, 1)
    df["season"] = np.where(df.month.isin([6, 7, 8]), 2, df["season"])
    df["season"] = pd.Series(np.where(df.month.isin([9, 10, 11]), 3, df["season"])).astype("int8")
    
    return df

# Assuming 'date' is a datetime column in your DataFrame 'd'
# Make sure 'date' is in the datetime format before applying the function
d['date'] = pd.to_datetime(d['date'])
d = create_date_features(d)


In [ ]:
## Lag Feature
## ACF & PACF
## PACF- Is very useful to decide which feature should we select 


In [ ]:
a = d[(d.sales.notnull())].groupby(["date", "family"]).sales.mean().reset_index().set_index("date")
for num, i in enumerate(a.family.unique()):
    try:
        fig, ax = plt.subplots(1,2,figsize=(15,5))
        temp = a[(a.family == i)]#& (a.sales.notnull())
        sm.graphics.tsa.plot_acf(temp.sales, lags=365, ax=ax[0], title = "AUTOCORRELATION\n" + i)
        sm.graphics.tsa.plot_pacf(temp.sales, lags=365, ax=ax[1], title = "PARTIAL AUTOCORRELATION\n" + i)
    except:
        pass

In [ ]:
a = d[d.year.isin([2016,2017])].groupby(["year", "day_of_year"]).sales.mean().reset_index()
px.line(a, x = "day_of_year", y = "sales", color = "year", title = "Average sales for 2016 and 2017")


In [ ]:
## Simple Moving  Average
a = train_df.sort_values(["store_nbr", "family", "date"])
for i in [20, 30, 45, 60, 90, 120, 365, 730]:
    a["SMA"+str(i)+"_sales_lag16"] = a.groupby(["store_nbr", "family"]).rolling(i).sales.mean().shift(16).values
    a["SMA"+str(i)+"_sales_lag30"] = a.groupby(["store_nbr", "family"]).rolling(i).sales.mean().shift(30).values
    a["SMA"+str(i)+"_sales_lag60"] = a.groupby(["store_nbr", "family"]).rolling(i).sales.mean().shift(60).values
print("Correlation")
a[["sales"]+a.columns[a.columns.str.startswith("SMA")].tolist()].corr()


In [ ]:
b = a[(a.store_nbr == 1)].set_index("date")
for i in b.family.unique():
    fig, ax = plt.subplots(2,4,figsize=(20,10))
    b[b.family == i][["sales", "SMA20_sales_lag16"]].plot(legend = True, ax = ax[0,0], linewidth = 4)
    b[b.family == i][["sales", "SMA30_sales_lag16"]].plot(legend = True, ax = ax[0,1], linewidth = 4)
    b[b.family == i][["sales", "SMA45_sales_lag16"]].plot(legend = True, ax = ax[0,2], linewidth = 4)
    b[b.family == i][["sales", "SMA60_sales_lag16"]].plot(legend = True, ax = ax[0,3], linewidth = 4)
    b[b.family == i][["sales", "SMA90_sales_lag16"]].plot(legend = True, ax = ax[1,0], linewidth = 4)
    b[b.family == i][["sales", "SMA120_sales_lag16"]].plot(legend = True, ax = ax[1,1], linewidth = 4)
    b[b.family == i][["sales", "SMA365_sales_lag16"]].plot(legend = True, ax = ax[1,2], linewidth = 4)
    b[b.family == i][["sales", "SMA730_sales_lag16"]].plot(legend = True, ax = ax[1,3], linewidth = 4)
    plt.suptitle("STORE 1 - "+i, fontsize = 15)
    plt.tight_layout(pad = 1.5)
    for j in range(0,4):
        ax[0,j].legend(fontsize="x-large")
        ax[1,j].legend(fontsize="x-large")
    plt.show()

#

In [ ]:
## Exponential Moving Average
def ewm_features(dataframe, alphas, lags):
    dataframe = dataframe.copy()
    for alpha in alphas:
        for lag in lags:
            dataframe['sales_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                dataframe.groupby(["store_nbr", "family"])['sales']. \
                    transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return dataframe

alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [16, 30, 60, 90]

a = ewm_features(a, alphas, lags)

In [ ]:
a[(a.store_nbr == 1) & (a.family == "GROCERY I")].set_index("date")[["sales", "sales_ewm_alpha_095_lag_16"]].plot(title = "STORE 1 - GROCERY I");